In [65]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import matplotlib.pyplot as plt


In [67]:
transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.Lambda(lambda img: img.convert("RGB")),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.Normalize([0.5, 0.5, 0.5], 
                         [0.5, 0.5, 0.5])
])

In [69]:
dataset = datasets.Caltech101(
    root="./data", 
    transform=transform, 
    download=False
)

In [71]:
print("Total images:", len(dataset))
print("Number of categories:", len(dataset.categories))
print("First 5 categories:", dataset.categories[:5])

Total images: 8677
Number of categories: 101
First 5 categories: ['Faces', 'Faces_easy', 'Leopards', 'Motorbikes', 'accordion']


In [73]:
# 80% train / 20% test
train_size = int(0.8 * len(dataset))
test_size  = len(dataset) - train_size


In [75]:
train_ds, test_ds = random_split(dataset, [train_size, test_size])

In [77]:
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=32, shuffle=False)

In [79]:
len(train_ds),len(test_ds)

(6941, 1736)

# Normal CNN architecture

In [81]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3,32,kernel_size=(3,3),padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2),stride=2),
            nn.Conv2d(32,64,kernel_size=(3,3),padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2),stride=2),

            nn.Flatten(),
            nn.Linear(64*32*32,256),
            nn.ReLU(),

            nn.Linear(256,128),
            nn.ReLU(),

            nn.Linear(128,len(dataset.categories))
        )

    def forward(self,x):
        return self.network(x)

In [83]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Initialize your model
model = CNN().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Using device: cuda


In [84]:
# Training function
def train_model(model, loader, optimizer, criterion, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(loader):.4f}")


In [87]:
# Evaluation function
def evaluate_model(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

In [89]:
# Run training
train_model(model, train_loader, optimizer, criterion, epochs=5)

Epoch [1/5], Loss: 3.3655
Epoch [2/5], Loss: 2.4692
Epoch [3/5], Loss: 1.9860
Epoch [4/5], Loss: 1.6434
Epoch [5/5], Loss: 1.3709


In [91]:
# Run evaluation
evaluate_model(model, test_loader)

Test Accuracy: 53.86%


# Transfer Learning

### Resnet18

In [96]:
from torchvision import models

# Loading ResNet18 with pretrained weights
model = models.resnet18(weights="DEFAULT")

# Freeze all convolutional layers
for param in model.parameters():
    param.requires_grad = False

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\Shrinivass/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100%|█████████████████████████████████████████████████████████████████████████████| 44.7M/44.7M [00:37<00:00, 1.24MB/s]


In [98]:
# Replacing the final layer (classifier) with one for our dataset
num_features = model.fc.in_features   # size of input to final layer
model.fc = nn.Linear(num_features, len(dataset.categories))  # Caltech101 has 101 classes

In [100]:
model = model.to(device)

In [102]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [104]:
# Train loop
def train_model(model, loader, optimizer, criterion, epochs=5):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        epoch_loss = running_loss / len(loader)
        epoch_acc = 100 * correct / total
        print(f"Epoch [{epoch+1}/{epochs}] Loss: {epoch_loss:.4f} Acc: {epoch_acc:.2f}%")


In [106]:
# Evaluation on Test Set
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")


In [108]:
train_model(model, train_loader, optimizer, criterion, epochs=5)

Epoch [1/5] Loss: 2.2669 Acc: 52.87%
Epoch [2/5] Loss: 0.9638 Acc: 78.03%
Epoch [3/5] Loss: 0.7246 Acc: 82.38%
Epoch [4/5] Loss: 0.6192 Acc: 84.34%
Epoch [5/5] Loss: 0.5394 Acc: 86.20%


In [110]:
evaluate(model, test_loader)

Test Accuracy: 80.07%


# EfficientNet-B0

In [114]:
# Loading pretrained EfficientNet-B0
model = models.efficientnet_b0(weights="DEFAULT")

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to C:\Users\Shrinivass/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-7f5810bc.pth


100%|█████████████████████████████████████████████████████████████████████████████| 20.5M/20.5M [00:17<00:00, 1.24MB/s]


In [116]:
# Freezing all layers
for param in model.parameters():
    param.requires_grad = False

In [118]:
# Replacing the classifier for Caltech101 (101 classes)
num_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_features, len(dataset.categories))


In [120]:
model = model.to(device)

In [122]:
criterion = nn.CrossEntropyLoss()

# Only train the classifier
optimizer = optim.Adam(model.classifier[1].parameters(), lr=0.001)

In [124]:
# Training loop

def train_model(model, loader, optimizer, criterion, epochs=5):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        epoch_loss = running_loss / len(loader)
        epoch_acc = 100 * correct / total
        print(f"Epoch [{epoch+1}/{epochs}] Loss: {epoch_loss:.4f} Acc: {epoch_acc:.2f}%")


In [126]:
# Evaluation

def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Test Accuracy: {100 * correct / total:.2f}%")


In [128]:
train_model(model, train_loader, optimizer, criterion, epochs=5)

Epoch [1/5] Loss: 2.3347 Acc: 53.35%
Epoch [2/5] Loss: 1.0659 Acc: 76.66%
Epoch [3/5] Loss: 0.8075 Acc: 81.75%
Epoch [4/5] Loss: 0.6884 Acc: 83.52%
Epoch [5/5] Loss: 0.5993 Acc: 85.84%


In [130]:
evaluate(model, test_loader)

Test Accuracy: 80.07%
